In [44]:
import tensorflow as tf 
from Vnet_multiclass import FillWithBone
from Unet_modified import CreateUnetModified
import os
from sklearn.model_selection import train_test_split
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt 
import skimage.transform as skTrans
import math

In [45]:
rootDirPath = "..\\mmwhs\\ct_train"
pathList = os.listdir(rootDirPath)

In [50]:
def OneHotEncode(original, x_, y_, z_, num_classes=8): 
    encodeList = {
        0: 0, 
        500: 1, 
        600: 2, 
        420: 3, 
        550: 4, 
        205: 5, 
        820: 6, 
        850: 7, 
    }

    x, y, z = original.shape

    labelEncode = np.empty((x_, y_, z_, num_classes), dtype="float16")
    for i_ in range(x_): 
        for j_ in range(y_): 
            for k_ in range(z_): 
                i = math.floor( (i_ * x) / x_)
                j = math.floor( (j_ * y) / y_)
                k = math.floor( (k_ * z) / z_)

                i = max(0, min(i, x - 1))
                j = max(0, min(j, y - 1))
                k = max(0, min(k, k - 1))

                value = original[i][j][k]

                encodeIndex = encodeList[value] = 1

                for n in range(8): 
                    labelEncode[i_][j_][k_][n] = 0

                labelEncode[i_][j_][k_][encodeIndex] = 1

    return labelEncode

In [ ]:
a = np.array(nib.load("E:\isef-vascular\VascuIAR\DeepLearning\mmwhs\ct_train\ct_train_1001_image.nii.gz").get_fdata())

with open(".\\temp.txt", "w") as f: 
    f.write("First view\n")
    for i in range(a.shape[0]): 
        u = np.unique(a[i, :, :])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

    f.write("\nSecond view\n")
    for i in range(a.shape[1]): 
        u = np.unique(a[:, i, :])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

    f.write("\nThird view\n")
    for i in range(a.shape[2]): 
        u = np.unique(a[:, :, i])
        f.write(f"Slice {i} has {len(u)} unique values which are: {u}\n")
    del i 

### EACH VIEW HAS DIFFERENT UNIQUE VALUES => INCONSISTENT WHEN TRYING TO REMOVE SLICES WITHOUT HEART + BY REMOVING SLICES WITHOUT HEART, IT WEAKENS THE MODEL'S ABILITY TO DIFFRENTIATE STRUCTUERS LIKE BONES / AIR IN SLICES CONTAINING HEART. 

# => 2 SOLUTIONS: 
#   1) Fill labels with zeros and real data with bone structures to make everything the same size
#   2) Crop all slices without heart to make it the same size  

# Cropping and Adding all add biases and unbalance to the dataset, except cropping is much harder to do homologously => Fill and offset bias 

In [ ]:
def GenerateData(start, stop, pathList):
  # target is 368 to allow Vnet to compress and decompress
  X = []
  Y = []
  for i in range(start, stop, 2):
      imagePath = os.path.join(rootDirPath, pathList[i])
      labelPath = os.path.join(rootDirPath, pathList[i + 1])

      image = np.array(nib.load(imagePath).get_fdata(), dtype="float16")
      label = np.array(nib.load(labelPath).get_fdata(), dtype="int8")

      image = skTrans.resize(image, (128, 128, 256), order=1, preserve_range=True)


      label = OneHotEncode(label, 128, 128, 256)

      image = np.expand_dims(image, -1)


      print(f"Image shape: {image.shape} || Path: {imagePath}")
      print(f"Label shape : {label.shape} || Path: {labelPath}")

      X.append(image)
      Y.append(label)

  X = np.array(X)
  Y = np.array(Y)

  del image
  del label
  del imagePath
  del labelPath

  return X, Y


def TrainingLoop(model):
  for i in range(0, 30, 2):
    X, Y = GenerateData(i, i + 2, pathList)
    model.fit(X, Y, epochs=1, verbose=1)
    del X
    del Y


In [ ]:
unet = CreateUnetModified()

In [ ]:
TrainingLoop(unet)